In [1]:
import os
import sys
if ".." not in sys.path:
    sys.path.append("..")
    os.chdir("..")

import gc
import datasets
import json
import string
import torch
import numpy as np
from glob import glob

from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from safetensors.torch import load_file, save_file, safe_open
from src import Data, Metrics, ModelArgs, get_model_and_tokenizer

args = ModelArgs()

/opt/tiger/miniconda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# v2 training/testing (save oracle ctr and search_id)

def process_fn(data):
    outputs = {"query": [], "pos": [], "neg": [], "pos_real_ctrs": [], "pos_oracle_ctrs": [], "neg_real_ctrs": [], "neg_oracle_ctrs": [], "search_id": [], "pos_ids": [], "neg_ids": []}
    for query, doc, search_result_click_cnt, real_ctr, oracle_ctr, search_id, doc_id in zip(data["query"], data["doc_info"], data["search_result_click_cnt"], data["real_clk_ctr"], data["video_oracle_ctr"], data["search_id"], data["doc_id"]):
        if doc is None:
            continue

        doc = json.loads(doc)
        real_ctr = float(real_ctr)
        oracle_ctr = float(oracle_ctr)

        doc_text = ""

        fields = [
            ("title", doc.get('title', '').strip()),
            ("username", doc.get('username', '').strip()),
            ("music", doc.get('music', '').strip()),
            ("poi", doc.get('poi', '').strip()),
            ("challenge", doc.get('challenge', '').strip()),
            ("ocr", doc.get('ocr', '').strip()),
            ("asr", doc.get('asr', '').strip())
        ]

        for field_name, field_value in fields:
            doc_text += f"<{field_name}>{field_value}\n\n"

        outputs["query"].append(query)
        outputs["search_id"].append(search_id)
        if search_result_click_cnt > 0:
            outputs["pos"].append([doc_text])
            outputs["neg"].append(["NAN"])
            outputs["pos_real_ctrs"].append([real_ctr])
            outputs["pos_oracle_ctrs"].append([oracle_ctr])
            outputs["pos_id"].append([doc_id])
            outputs["neg_real_ctrs"].append(None)
            outputs["neg_oracle_ctrs"].append(None)
        else:
            outputs["pos"].append(["NAN"])
            outputs["neg"].append([doc_text])
            outputs["pos_real_ctrs"].append(None)
            outputs["pos_oracle_ctrs"].append(None)
            outputs["neg_real_ctrs"].append([real_ctr])
            outputs["neg_oracle_ctrs"].append([oracle_ctr])

        # else:
        #     raise ValueError(f"Fuck query {query} search_cnt {search_result_click_cnt}")


    return outputs

base_dir = "/mnt/bn/search-douyin-rank-yg/all_data_from_lf/text_embedding_data/query_doc_info_sample_order_1102_eval_v1"
# base_dir = "/mnt/bn/search-douyin-rank-yg/all_data_from_lf/text_embedding_data/query_doc_info_sample_order_1102_eval_v2"
# base_dir = "/mnt/bn/search-douyin-rank-yg/all_data_from_lf/text_embedding_data/query_doc_info_sample_order_1102_eval_v3"
dataset = datasets.load_dataset("parquet", data_files=glob(f"{base_dir}/*.parquet"), split="train")
# contra_dataset = dataset.map(process_fn, num_proc=32, batched=True, remove_columns=dataset.column_names, batch_size=100)
# contra_dataset.save_to_disk(f"{base_dir}_peitian")

Generating train split: 1290334 examples [00:14, 86871.90 examples/s]
